# GPU-accelerated image processing using CUPY and CUCIM
Processing large images with python can take time. In order to accelerate processing, graphics processing units (GPUs) can be exploited, for example using [NVidia CUDA](https://en.wikipedia.org/wiki/CUDA). For processing images with CUDA, there are a couple of libraries available. We will take a closer look at [cupy](https://cupy.dev/), which brings more general computing capabilities for CUDA compatible GPUs, and [cucim](https://github.com/rapidsai/cucim), a library of image processing specific operations using CUDA. Both together can serve as GPU-surrogate for [scikit-image](https://scikit-image.org/).

See also
* [StackOverflow: Is it possible to install cupy on google colab?](https://stackoverflow.com/questions/49135065/is-it-possible-to-install-cupy-on-google-colab)
* [Cucim example notebooks](https://github.com/rapidsai/cucim/blob/branch-0.20/notebooks/Welcome.ipynb)

Before we start, we need to install CUDA and CUCIM it properly. The following commands make this notebook run in Google Colab.

In [ ]:
!curl https://colab.chainer.org/install | sh -

!pip install cucim
!pip install scipy scikit-image cupy-cuda100

In [ ]:
import numpy as np
import cupy as cp
import cucim
from skimage.io import imread, imshow
import pandas as pd

In the following, we are using image data from Paci et al shared under  the [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license. See also: https://doi.org/10.17867/10000140 


In [ ]:
image = imread('https://idr.openmicroscopy.org/webclient/render_image_download/9844418/?format=tif')

imshow(image)

In order to process an image using CUDA on the GPU, we need to convert it. Under the hood of this conversion, the image data is sent from computer random access memory (RAM) to the GPUs memory.

In [ ]:
image_gpu = cp.asarray(image)

image_gpu.shape

Extracting a single channel out of the three-channel image works like if we were working with numpy. Showing the image does not work, because the CUDA image is not available in memory. In order to get it back from GPU memory, we need to convert it to a numpy array.

In [ ]:
single_channel_gpu = image_gpu[:,:,1]

# the following line would fail
# imshow(single_channel_gpu)

# get single channel image back from GPU memory and show it
single_channel = cp.asnumpy(single_channel_gpu)
imshow(single_channel)

# we can also do this with a convenience function
def gpu_imshow(image_gpu):
  image = np.asarray(image_gpu)
  imshow(image)

## Image filtering and segmentation

The cucim developers have re-implemented many functions from sckit image, e.g. the [Gaussian blur filter](https://docs.rapids.ai/api/cucim/stable/api.html#cucim.skimage.filters.gaussian), [Otsu Thresholding](https://docs.rapids.ai/api/cucim/stable/api.html#cucim.skimage.filters.threshold_otsu) after [Otsu et al. 1979](https://ieeexplore.ieee.org/document/4310076), [binary erosion](https://docs.rapids.ai/api/cucim/stable/api.html#cucim.skimage.morphology.binary_erosion) and [connected component labeling](https://docs.rapids.ai/api/cucim/stable/api.html#cucim.skimage.measure.label).

In [ ]:
 from cucim.skimage.filters import gaussian

 blurred_gpu = gaussian(single_channel_gpu, sigma=5)

 gpu_imshow(blurred_gpu)

In [ ]:
from cucim.skimage.filters import threshold_otsu

# determine threshold
threshold = threshold_otsu(blurred_gpu)

# binarize image by apply the threshold
binary_gpu = blurred_gpu > threshold

gpu_imshow(binary_gpu)

In [ ]:
from cucim.skimage.morphology import binary_erosion, disk

eroded_gpu = binary_erosion(binary_gpu, selem=disk(2))

gpu_imshow(eroded_gpu)

In [ ]:
from cucim.skimage.measure import label

labels_gpu = label(eroded_gpu)

gpu_imshow(labels_gpu)

For visualization purposes, it is recommended to turn the label image into an RGB image, especially if you want to save it to disk.

In [ ]:
from cucim.skimage.color import label2rgb

labels_rgb_gpu = label2rgb(labels_gpu)

gpu_imshow(labels_rgb_gpu)

## Quantitative measurements

Also quantitative measurments using [regionprops_table](https://docs.rapids.ai/api/cucim/stable/api.html#cucim.skimage.measure.regionprops_table) have been implemented in cucim. A major difference is that you need to convert its result back to numpy if you want to continue processing on the CPU, e.g. using [pandas](https://pandas.pydata.org/).

In [ ]:
from cucim.skimage.measure import regionprops_table 

table_gpu = regionprops_table(labels_gpu, intensity_image=single_channel_gpu, properties=('mean_intensity', 'area', 'solidity'))

table_gpu

In [ ]:
# The following line would fail.
# pd.DataFrame(table_gpu)

# We need to convert that table to numpy before we can pass it to pandas.
table = {item[0] : cp.asnumpy(item[1]) for item in table_gpu.items()}
pd.DataFrame(table)

# GPU libraries

In [22]:
import numpy as np

data_a = np.random.random([100,1])
data_b = np.random.random([100,1])

mat_c = np.matmul(data_a, data_b.T)
mat_c.shape

(100, 100)

In [25]:
import cupy as cp

cp_data_a = cp.asarray(data_a)
cp_data_b = cp.asarray(data_b)

cp_mat_c = cp.matmul(cp_data_a, cp_data_b.T)
cp_mat_c.shape


(100, 100)

## pyopencl

In [5]:
import pyopencl as cl
import pyopencl.array as cla


context = cl.create_some_context()
queue = cl.CommandQueue(context)

a = cla.to_device(queue, np.asarray([1, 2, 3]))
b = cla.to_device(queue, np.asarray([4, 4, 5]))  

c = a + b

print(c)

[5 6 8]


In [6]:
d = cla.empty_like(a)

program = cl.Program(context, """
__kernel void sum(__global const float *a, __global const float *b, __global float *c)
{
  int i = get_global_id(0);
  c[i] = a[i] + b[i];
}""").build()

program.sum(queue, a.shape, None, a.data, b.data, d.data)

print(d)

[5 6 8]


## Image processing

In [ ]:
from skimage.data import cells3d

stack = cells3d()

In [ ]:
stack.shape

In [ ]:
from skimage.io import imshow

imshow(stack[30, 1, :, :])

In [ ]:
single_channel = stack[:, 1, :, :]
single_channel.shape

In [ ]:
import skimage

def segment_nuclei(nuclei_image):
  input_image = nuclei_image

  # binarize all nuclei together
  outline_blurred = skimage.filters.gaussian(input_image, sigma=2)
  threshold = skimage.filters.threshold_otsu(outline_blurred)
  binary = outline_blurred > threshold

  # estimate centroids
  spot_blurred = skimage.filters.gaussian(input_image, sigma=10)
  local_maxima = skimage.feature.peak_local_max(
      spot_blurred, 
      footprint=np.ones((3, 3, 3), dtype=np.bool), 
      indices=False, 
      labels=skimage.measure.label(binary)
  )

  # flood the binary image starting at the centroids
  markers = skimage.measure.label(local_maxima)
  labels = skimage.segmentation.watershed(-spot_blurred, markers, mask=binary)
  
  return labels

nuclei = segment_nuclei(single_channel)
imshow(nuclei.max(axis=0))

# CUDA

In [ ]:
import cupy
cupy.cuda.runtime.getDeviceProperties(0)['name']

In [ ]:
import cucim.skimage as cuskimage

def cu_segment_nuclei(nuclei_image):
  # send image to GPU
  input_image = cupy.asarray(nuclei_image)

  # binarize all nuclei together
  outline_blurred = cuskimage.filters.gaussian(input_image, sigma=2)
  threshold = cuskimage.filters.threshold_otsu(outline_blurred)
  binary = outline_blurred > threshold

  # estimate centroids
  spot_blurred = cuskimage.filters.gaussian(input_image, sigma=10)
  local_maxima = cuskimage.feature.peak_local_max(
      spot_blurred, 
      footprint=np.ones((3, 3, 3), dtype=np.bool), 
      indices=False, 
      labels=cuskimage.measure.label(binary)
  )

  # flood the binary image starting at the centroids
  markers = cuskimage.measure.label(local_maxima)
  labels = cuskimage.segmentation.watershed(-spot_blurred, markers, mask=binary)
  
  return np.asarray(labels)

nuclei = cu_segment_nuclei(single_channel)
imshow(nuclei.max(axis=0))

# OpenCL

## pyclesperanto

In [ ]:
!pip install pyclesperanto_prototype

In [7]:
import pyclesperanto_prototype as cle
cle.get_device()

<gfx902 on Platform: AMD Accelerated Parallel Processing (2 refs)>

In [9]:
def segment_nuclei(nuclei_image):
  input_image = nuclei_image

  # binarize all nuclei together
  outline_blurred = cle.gaussian_blur(input_image, sigma_x=2, sigma_y=2, sigma_z=2)
  binary = cle.threshold_otsu(outline_blurred)
  
  # estimate centroids
  spot_blurred = cle.gaussian_blur(input_image, sigma_x=10, sigma_y=10, sigma_z=10)
  local_maxima = cle.detect_maxima_box(spot_blurred, radius_x=1, radius_y=1, radius_z=1)

  # flood the binary image starting at the centroids
  labels = cle.masked_voronoi_labeling(local_maxima, binary)
  
  return labels

nuclei = segment_nuclei(single_channel)
imshow(nuclei.max(axis=0))

NameError: name 'single_channel' is not defined

In [8]:
cle.masked_voronoi_labeling?